In [39]:
import os
import pandas as pd
import pickle
import json
import requests
import ast

import turicreate
from turicreate import SFrame

import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model, InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import AciWebservice, LocalWebservice, Webservice,  AksWebservice
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

In [40]:
print("AZML SDK Version:", azureml.core.VERSION)
print("Pandas Version:", pd.__version__)

AZML SDK Version: 1.48.0
Pandas Version: 1.1.5


In [41]:
# If in Databricks
# OUTPUT_DIR = "/dbfs/FileStore/RetailRecommender/output/"
# MODELS_DIR = "/dbfs/FileStore/RetailRecommender/models/"
# DEPLOY_DIR = "/dbfs/FileStore/RetailRecommender/deploy/"

OUTPUT_DIR = "./output/"
MODELS_DIR = "./models/"
DEPLOY_DIR = "./deploy/"

os.makedirs(DEPLOY_DIR, exist_ok=True)

# 1. Load the AML Workspace

In [42]:
ws = Workspace(workspace_name="RRBrazilex", 
                subscription_id="2f8a2563-4e90-4a88-8e6c-56fa0f12cb7e", 
                resource_group="retail-rec-brazil-ex")

# ws = Workspace(workspace_name="YOUR_WORKSPACE_NAME", 
#                 subscription_id="YOUR_SUBSCRIPTION_ID", 
#                 resource_group="YOUR_RESOURCE_GROUP")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = "\n")

RRBrazilex
retail-rec-brazil-ex
eastus
2f8a2563-4e90-4a88-8e6c-56fa0f12cb7e


# 2. Register the Models on AZML service model registry

In [43]:
model_s_path = MODELS_DIR + "recommendation_s.model"
model_r_path = MODELS_DIR + "recommendation_r.model"

In [44]:
azmodel_s = Model.register( model_path = model_s_path,
                           model_name = "similarity_model",
                           workspace = ws)

azmodel_r = Model.register( model_path = model_r_path,
                           model_name = "ranking_factorization_model",
                           workspace = ws)

Registering model similarity_model
Registering model ranking_factorization_model


# 3. Test score.py init() and run() functions

In [45]:
import os

def init():
    
    global model_s
    global model_r
    global product_column_name
    global customer_column_name
    global reviewscore_column_name
    
    order_column_name = "order_id"
    product_column_name = "product_id"
    timestamp_column_name = "order_purchase_timestamp"
    category_column_name = "product_category_name_english"
    reviewscore_column_name = "review_score"
    customer_column_name = "customer_unique_id"
    customer_city_column_name = "customer_city"
    
    model_s = turicreate.load_model(model_s_path)
    model_r = turicreate.load_model(model_r_path)
    

In [73]:
def run(raw_data):
    
    data = json.loads(raw_data)
    data = pd.DataFrame(data).to_dict(orient="list")

    try:
        if customer_column_name in data:
            print(customer_column_name + " found")
            
            # If observation side data exists
            if ("period" or "month" or "weekday") in data:
                print("Using Ranking Factorization Model\n")
                users_query = turicreate.SFrame(
                    {"customer_unique_id": data[customer_column_name], 
                    "period": data["period"] if "period" in data else ["Night"] * len(data[customer_column_name]),
                    "month": data["month"] if "month" in data else [8] * len(data[customer_column_name]),
                    "weekday": data["weekday"] if "weekday" in data else [2] * len(data[customer_column_name])
                    }
                )

                result = model_r.recommend(users=users_query, k=10).to_dataframe()
                
            else:
                print("Using Similarity Model\n")
                result = model_s.recommend(users=data[customer_column_name], k=10).to_dataframe()
                
            if product_column_name in data:
                return "Error: Request cannot have" + product_column_name + " and" + customer_column_name + " together. Please make separate requests for user recommendations and for items basket similarity"
        
        elif product_column_name in data:
                print("Using Similarity Model\n")
                result = model_s.get_similar_items(data[product_column_name], k=10).to_dataframe()
                
        else:
            return "Error: Payload must contain at least a field named " + customer_column_name + " or " + product_column_name
    
        return result.to_json(orient="records")
        
    except Exception as e:
        return e


In [74]:
init()

In [75]:

test_load1 = '''
    [
        {"customer_unique_id":"871766c5855e863f6eccc05f988b23cb", "period":"Night",  "weekday":2},
        {"customer_unique_id":"871766c5855e863f6eccc05f988b23cb", "period":"Night", "weekday":2}
    ]
'''

test_load2 = '''
    [
        {"customer_unique_id":"871766c5855e863f6eccc05f988b23cb"},
        {"customer_unique_id":"871766c5855e863f6eccc05f988b23cb"}
    ]
'''

test_load3 = '''
    [
        {"customer_unique_id":"871766c5855e863f6eccc05f988b23cb", "period":"Night",  "weekday":2 , "month": 9},
        {"customer_unique_id":"871766c5855e863f6eccc05f988b23cb", "period":"Night",  "weekday":2 , "month": 9}
    ]
'''

test_load4 = '''
    [
        {"product_id":"4244733e06e7ecb4970a6e2683c13e61"},
        {"product_id":"fdcf45aa23bb8312ecc0027d6e1ef1c4"}
    ]
'''


In [76]:
run(test_load3)

customer_unique_id found
Using Ranking Factorization Model



'[{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"aca2eb7d00ea1a7b8ebd4e68314663af","score":4.6170641054,"rank":1},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"99a4788cb24856965c36a24e339b6058","score":4.1736911464,"rank":2},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"186fe07c8b8005ec6a498587ffbc1352","score":4.1233484102,"rank":3},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"2136c70bbe723d338fab53da3c03e6dc","score":4.028887212,"rank":4},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"a62e25e09e05e6faf31d90c6ec1aa3d1","score":4.0174223245,"rank":5},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"777d2e438a1b645f3aec9bd57e92672c","score":3.9907904232,"rank":6},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"422879e10f46682990de24d770e7f83d","score":3.9425410001,"rank":7},{"customer_unique_id":"871766c5855e863f6eccc05f

In [77]:
run(test_load4)

Using Similarity Model



'[]'

# 4. Create Inference Environment

In [78]:
# curated_env = Environment.get(workspace=ws, name="AzureML-minimal-ubuntu18.04-py37-cpu-inference")
# myenv = curated_env.clone("turicreate_env")

conda_dep = CondaDependencies()
conda_dep.add_pip_package("turicreate")

myenv = Environment(name="turicreate_env")
myenv.python.conda_dependencies=conda_dep
myenv.inferencing_stack_version = "latest"

# Register the Environment on AZML so we can retreive it later
myenv = myenv.register(workspace=ws)

# Uncomment this to re-build the environment
# myenv.build(workspace=ws)


# 5. Create the inference configuration

### !!! Make sure that in score.py you set the right model version!!!

In [81]:
scorepy_text = '''
import os
import sys
import turicreate
from turicreate import SFrame
import pandas as pd
import json

def init():
    
    global model_s
    global model_r
    
    model_s_name = "similarity_model"
    model_s_version = "{S_VERSION}"
    model_s_filename = "recommendation_s.model"
    
    model_r_name = "ranking_factorization_model"
    model_r_version = "{R_VERSION}"  
    model_r_filename = "recommendation_r.model"
    
    model_s_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_s_name, model_s_version, model_s_filename)
    model_r_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_r_name, model_r_version, model_r_filename)
    
    print("model_s_path",model_s_path)
    print("model_r_path",model_r_path)
    
    model_s = turicreate.load_model(model_s_path)
    model_r = turicreate.load_model(model_r_path)
    
    
def run(raw_data):
    
    data = json.loads(raw_data)
    data = pd.DataFrame(data).to_dict(orient="list")

    try:
        if "{USER_ID}" in data:
            print("{USER_ID} found")
            
            # If observation side data exists
            if ("period" or "month" or "weekday") in data:
                print("Using Ranking Factorization Model")
                users_query = turicreate.SFrame(
                    {{"customer_unique_id": data["{USER_ID}"], 
                    "period": data["period"] if "period" in data else ["Night"] * len(data["{USER_ID}"]),
                    "month": data["month"] if "month" in data else [8] * len(data["{USER_ID}"]),
                    "weekday": data["weekday"] if "weekday" in data else [2] * len(data["{USER_ID}"])
                    }}
                )
                result = model_r.recommend(users=users_query, k=10).to_dataframe()
                
            else:
                print("Using Similarity Model")
                result = model_s.recommend(users=data["{USER_ID}"], k=10).to_dataframe()
                
            if "{ITEM_ID}" in data:
                return "Error: Request cannot have {USER_ID} and {ITEM_ID} together. Please make separate requests for {USER_ID} recommendations and for {ITEM_ID} basket similarity"
        
        elif "{ITEM_ID}" in data:
                print("Using Similarity Model")
                result = model_s.get_similar_items(data["{ITEM_ID}"], k=10).to_dataframe()
                
        else:
            return "Error: Payload must contain at least a field named {USER_ID} or {ITEM_ID}"
    
        return result.to_json(orient="records")
        
    except Exception as e:
        return e

'''.format(S_VERSION=3, R_VERSION=3, USER_ID=customer_column_name, ITEM_ID=product_column_name)

with open(DEPLOY_DIR+"score.py", "w") as file:
    file.write(scorepy_text)

print("score.py written, using",DEPLOY_DIR+"score.py")

score.py written, using ./deploy/score.py


In [82]:
!cat {DEPLOY_DIR}score.py


import os
import sys
import turicreate
from turicreate import SFrame
import pandas as pd
import json

def init():
    
    global model_s
    global model_r
    
    model_s_name = "similarity_model"
    model_s_version = "3"
    model_s_filename = "recommendation_s.model"
    
    model_r_name = "ranking_factorization_model"
    model_r_version = "3"  
    model_r_filename = "recommendation_r.model"
    
    model_s_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_s_name, model_s_version, model_s_filename)
    model_r_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), model_r_name, model_r_version, model_r_filename)
    
    print("model_s_path",model_s_path)
    print("model_r_path",model_r_path)
    
    model_s = turicreate.load_model(model_s_path)
    model_r = turicreate.load_model(model_r_path)
    
    
def run(raw_data):
    
    data = json.loads(raw_data)
    data = pd.DataFrame(data).to_dict(orient="list")

    try:
        if "customer_unique_id" in data:
      

# 6 . Deploy to Local
This does not work on Databricks

In [83]:
inference_config = InferenceConfig(entry_script=DEPLOY_DIR+"score.py",environment=myenv)

deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, "test", [azmodel_s, azmodel_r], inference_config, deployment_config)

/tmp/ipykernel_64236/3062853723.py:5: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  local_service = Model.deploy(ws, "test", [azmodel_s, azmodel_r], inference_config, deployment_config)


Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 8cd100591dc0495bba6b05e638847d61.azurecr.io
Logging into Docker registry 8cd100591dc0495bba6b05e638847d61.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 8cd100591dc0495bba6b05e638847d61.azurecr.io/azureml/azureml_e09ca68920848b36c7b56c609f002bda
 ---> 76721a88f05f
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 1b2ded63a86d
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjJmOGEyNTYzLTRlOTAtNGE4OC04ZTZjLTU2ZmEwZjEyY2I3ZSIsInJlc291cmNlR3JvdXBOYW1lIjoicmV0YWlsLXJlYy1icmF6aWwtZXgiLCJhY2NvdW50TmFtZSI6InJyYnJhemlsZXgiLCJ3b3Jrc3BhY2VJZCI6IjhjZDEwMDU5LTFkYzAtNDk1Yi1iYTZiLTA1ZTYzODg0N2Q2MSJ9LCJtb2RlbHMiOnt9LCJtb2RlbHNJbmZvIjp7fX0= | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 3f9467b55459
 ---> f6596f1fb476
Step 4/5 : RUN mv '/var/azureml-app/tmpvf3hj_nw.py' /var/azureml-app/main.py
 ---> Running in d50

In [ ]:
# If container fails, check out the logs
# local_service.get_logs()

## Test local web service container

In [84]:
print("Local service hosted in:", local_service.scoring_uri)

Local service hosted in: http://localhost:6789/score


In [85]:
import json

response = local_service.run(input_data = test_load3)
print(response)

Checking container health...
Local webservice is running at http://localhost:6789
[{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"aca2eb7d00ea1a7b8ebd4e68314663af","score":4.6170641054,"rank":1},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"99a4788cb24856965c36a24e339b6058","score":4.1736911464,"rank":2},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"186fe07c8b8005ec6a498587ffbc1352","score":4.1233484102,"rank":3},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"2136c70bbe723d338fab53da3c03e6dc","score":4.028887212,"rank":4},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"a62e25e09e05e6faf31d90c6ec1aa3d1","score":4.0174223245,"rank":5},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"777d2e438a1b645f3aec9bd57e92672c","score":3.9907904232,"rank":6},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"422879e10f46682990de24d770e7f83

In [86]:
import urllib
import requests
import ast

# The URL will need to the edited if ACI or AKS service
url_aci = local_service.scoring_uri

headers = {'Content-Type':'application/json'}
body = test_load2

req = urllib.request.Request(url_aci, str.encode(body), headers)
response = ast.literal_eval(urllib.request.urlopen(req).read().decode('utf-8'))

print(response)


[{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"154e7e31ebfa092203795c972e5804a6","score":0.0067212236,"rank":1},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"36f60d45225e60c7da4558b070ce4b60","score":0.0060527384,"rank":2},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"2b4609f8948be18874494203496bc318","score":0.0043835616,"rank":3},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"e53e557d5a159f5aa2c5e995dfdf244b","score":0.0032941175,"rank":4},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"b1acb7e8152c90c9619897753a75c973","score":0.0032536769,"rank":5},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"3f14d740544f37ece8a9e7bc8349797e","score":0.002758621,"rank":6},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"7c1bd920dbdf22470b68bde975dd3ccf","score":0.002337662,"rank":7},{"customer_unique_id":"871766c5855e863f6eccc05f98

# 7. Deploy to Azure Container Instance

In [87]:
inference_config = InferenceConfig(entry_script=DEPLOY_DIR+"score.py",environment=myenv)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

aci_service = Model.deploy(ws, "aci-service", [azmodel_s, azmodel_r], inference_config, deployment_config, overwrite=True)

aci_service.wait_for_deployment(show_output=True)

/tmp/ipykernel_64236/1052589028.py:5: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  aci_service = Model.deploy(ws, "aci-service", [azmodel_s, azmodel_r], inference_config, deployment_config, overwrite=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-02-03 19:46:04+00:00 Creating Container Registry if not exists.
2023-02-03 19:46:05+00:00 Registering the environment.
2023-02-03 19:46:05+00:00 Use the existing image.
2023-02-03 19:46:05+00:00 Generating deployment configuration.
2023-02-03 19:46:08+00:00 Submitting deployment to compute.
2023-02-03 19:46:12+00:00 Checking the status of deployment aci-service..
2023-02-03 19:50:11+00:00 Checking the status of inference endpoint aci-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


## Test ACI URL API Endpoint

In [88]:
aci_service.scoring_uri

'http://f2b2eff3-4b12-4f4f-9273-7de5852f0a36.eastus.azurecontainer.io/score'

In [89]:
import urllib
import requests
import ast

# The URL will need to the edited if ACI or AKS service
url_aci = aci_service.scoring_uri

headers = {'Content-Type':'application/json'}
body = test_load2

req = urllib.request.Request(url_aci, str.encode(body), headers)
response = ast.literal_eval(urllib.request.urlopen(req).read().decode('utf-8'))

print(response)


[{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"154e7e31ebfa092203795c972e5804a6","score":0.0067212236,"rank":1},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"36f60d45225e60c7da4558b070ce4b60","score":0.0060527384,"rank":2},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"2b4609f8948be18874494203496bc318","score":0.0043835616,"rank":3},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"e53e557d5a159f5aa2c5e995dfdf244b","score":0.0032941175,"rank":4},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"b1acb7e8152c90c9619897753a75c973","score":0.0032536769,"rank":5},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"3f14d740544f37ece8a9e7bc8349797e","score":0.002758621,"rank":6},{"customer_unique_id":"871766c5855e863f6eccc05f988b23cb","product_id":"7c1bd920dbdf22470b68bde975dd3ccf","score":0.002337662,"rank":7},{"customer_unique_id":"871766c5855e863f6eccc05f98

In [90]:
aci_service.delete()

# 8. Deploy to Production -  managed AKS Cluster

Details for SSL and more best practices for production here: https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/production-deploy-to-aks/production-deploy-to-aks.ipynb

In [91]:
# Choose a name for your AKS cluster
aks_name = 'my-aks'

# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Use the default configuration (can also provide parameters to customize)
    # https://learn.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.aks.akscompute?view=azure-ml-py
    prov_config = AksCompute.provisioning_configuration()

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                    name = aks_name, 
                                    provisioning_configuration = prov_config)

if aks_target.get_status() != "Succeeded":
    aks_target.wait_for_completion(show_output=True)

InProgress........................................................................................................................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"


In [ ]:
# Set the web service configuration (using default here)
# https://learn.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.akswebservice?view=azure-ml-py#azureml-core-webservice-akswebservice-deploy-configuration
aks_config = AksWebservice.deploy_configuration()

# # Enable token auth and disable (key) auth on the webservice
# aks_config = AksWebservice.deploy_configuration(token_auth_enabled=True, auth_enabled=False)

In [ ]:
%%time

aks_service_name ='aks-service'

aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models=[azmodel_s, azmodel_r],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target,
                           overwrite=True)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

# 8. Test the web service using raw HTTP request

In [ ]:
# # if (key) auth is enabled, retrieve the API keys. AML generates two keys.
key1, Key2 = aks_service.get_keys()
print(key1)

# # if token auth is enabled, retrieve the token.
# access_token, refresh_after = aks_service.get_token()

In [ ]:
print(aks_service.scoring_uri)

In [ ]:
apiurl = aks_service.scoring_uri
key = key1
# key = "G46wSFtiiAFlCHHi0beyU29ceaB2dPRS"
# apiurl = "http://20.114.104.171:80/api/v1/service/aks-service/score"

In [ ]:
%%time

# # If (key) auth is enabled, don't forget to add key to the HTTP header.
headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key}

# # If token auth is enabled, don't forget to add token to the HTTP header.
# headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + access_token}

resp = requests.post(apiurl, bytes(test_load1,encoding = 'utf8'), headers=headers)

print(ast.literal_eval(resp.text))

In [ ]:
aks_service.delete()

In [ ]:
# aks_service.get_logs()